In [ ]:
import tensorflow as tf

# Caricamento del dataset (già suddiviso in train e test)
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist

# Suddivisione in training e validation set
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

X_train = X_train / 255.0
X_valid = X_valid / 255.0
X_test = X_test / 255.0

class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
"Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

class_names[y_train[0]]

In [ ]:
tf.random.set_seed(42)

model = tf.keras.Sequential([
tf.keras.Input(shape=(28, 28)),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(300, activation="relu"),
tf.keras.layers.Dense(100, activation="relu"),
tf.keras.layers.Dense(10, activation="softmax")
])

model.summary()

In [ ]:
from tensorflow.keras.optimizers import SGD
model.compile(
loss="sparse_categorical_crossentropy",
optimizer=SGD(learning_rate=0.01),
metrics=["accuracy"]
)

In [ ]:
history = model.fit(
X_train, y_train,
epochs=30,
validation_data=(X_valid, y_valid)
)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt

#Caricamento dataset
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

#Normalization: adattamento PRIMA del grafo
normalization_layer = layers.Normalization()
normalization_layer.adapt(X_train)

#Layer e struttura manuale
hidden1 = layers.Dense(30, activation="relu")
hidden2 = layers.Dense(30, activation="relu")
concat_layer = layers.Concatenate()
outputlayer = layers.Dense(1)

#Input simbolico
input = layers.Input(shape=X_train.shape[1:])  # es. 8 feature

#Costruzione del grafo computazionale
normalized = normalization_layer(input)       # normalizzazione (dopo adapt)
h1 = hidden1(normalized)                       # primo hidden layer
h2 = hidden2(h1)                               # secondo hidden layer
concat = concat_layer([normalized, h2])        # percorso wide + deep
output = outputlayer(concat)                  # output finale

#Costruzione modello
model = keras.Model(inputs=[input], outputs=[output])

#Compilazione
model.compile(
    loss="mse",
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=["RootMeanSquaredError"]
)

#Addestramento
history1 = model.fit(
    X_train, y_train,
    epochs=40,
    validation_data=(X_valid, y_valid)
)

#Plot
pd.DataFrame(history1.history).plot(
    figsize=(8, 5), xlim=[0, 40], ylim=[0, 3], grid=True,
    xlabel="Epoch", style=["r--", "r--.", "b-", "b-*"]
)
plt.title("Learning Curves")
plt.show()

In [ ]:
# Import delle librerie principali
import tensorflow as tf
import keras_tuner as kt
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from pathlib import Path
from time import strftime

# Funzione per generare automaticamente la directory dei log di TensorBoard
def get_run_logdir(root_logdir="my_logs"):
    return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S")

# Imposta la directory dei log per TensorBoard
run_logdir = get_run_logdir()

# Caricamento del dataset California Housing
housing = fetch_california_housing()

# Suddivisione in training, validation e test set
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

# Standardizzazione delle feature (molto importante per le reti neurali)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

# Funzione che costruisce dinamicamente il modello, ricevendo un oggetto hp (HyperParameters)
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=X_train.shape[1:]))

    # Costruisce un numero variabile di layer nascosti
    for i in range(hp.Int("num_layers", 1, 5)):
        model.add(tf.keras.layers.Dense(
            units=hp.Int(f"units_{i}", min_value=16, max_value=128, step=16),
            activation="relu"))

    # Layer di output per regressione (1 neurone, senza attivazione)
    model.add(tf.keras.layers.Dense(1))

    # Ottimizzatore Adam con tasso di apprendimento variabile
    optimizer = tf.keras.optimizers.Adam(
        learning_rate=hp.Float("lr", 1e-4, 1e-2, sampling="log")
    )

    # Compilazione del modello
    model.compile(
        loss="mse",
        optimizer=optimizer,
        metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )
    return model

# Callback di TensorBoard per registrare log durante l'addestramento
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=run_logdir, profile_batch=(100, 200))

# Tuner per ricerca bayesiana degli iperparametri
tuner = kt.BayesianOptimization(
    build_model,  # Funzione di costruzione del modello
    objective="val_root_mean_squared_error",  # Obiettivo da minimizzare
    max_trials=10,  # Numero massimo di combinazioni da testare
    directory="kt_dir",  # Directory dove salvare i log dei trial
    project_name="bayes_housing",  # Nome del progetto
    overwrite=True,  # Sovrascrive risultati precedenti
)

# Ricerca dei migliori iperparametri su 10 trial
tuner.search(
    X_train, y_train,
    epochs=10,
    validation_data=(X_valid, y_valid),
    callbacks=[tensorboard_cb]  # Integrazione di TensorBoard
)

# Recupero del miglior modello trovato
best_model = tuner.get_best_models(1)[0]

# Visualizzazione degli iperparametri migliori
best_hp = tuner.get_best_hyperparameters(1)[0]
print("Migliori iperparametri trovati:", best_hp.values)

# Addestramento finale del miglior modello su tutto il dataset (train + valid)
X_full = np.vstack([X_train, X_valid])
y_full = np.hstack([y_train, y_valid])
best_model.fit(X_full, y_full, epochs=40)

# Valutazione finale sul test set
test_loss, test_rmse = best_model.evaluate(X_test, y_test)
print("Test RMSE finale:", test_rmse)